In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical

# Load and preprocess the dataset
def load_and_preprocess_data():
    # Load data
    X_train = np.loadtxt("../../Datasets/CAPP Dataset/SubjectDependent50PercentOverlap/X_train.txt")
    y_train = np.loadtxt("../../Datasets/CAPP Dataset/SubjectDependent50PercentOverlap/y_train.txt")
    X_test = np.loadtxt("../../Datasets/CAPP Dataset/SubjectDependent50PercentOverlap/X_test.txt")
    y_test = np.loadtxt("../../Datasets/CAPP Dataset/SubjectDependent50PercentOverlap/y_test.txt")

    # Reshape data (assuming 100 time steps and 9 features)
    X_train = X_train.reshape(X_train.shape[0], 100, 9)
    X_test = X_test.reshape(X_test.shape[0], 100, 9)

    # Normalize features
    scaler = StandardScaler()
    X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])
    X_test_reshaped = X_test.reshape(-1, X_test.shape[-1])
    X_train_scaled = scaler.fit_transform(X_train_reshaped).reshape(X_train.shape)
    X_test_scaled = scaler.transform(X_test_reshaped).reshape(X_test.shape)

    # Manually flatten the input data
    X_train_flattened = X_train_scaled.reshape(X_train_scaled.shape[0], -1)  # (samples, 900)
    X_test_flattened = X_test_scaled.reshape(X_test_scaled.shape[0], -1)     # (samples, 900)

    # Find the actual number of unique classes
    all_labels = np.unique(np.concatenate((y_train, y_test)))
    num_classes = len(all_labels)
    print(f"Unique labels: {all_labels}")
    print(f"Number of classes: {num_classes}")

    # Create a mapping from original labels to consecutive integers
    label_map = {label: i for i, label in enumerate(all_labels)}

    # Apply the mapping to y_train and y_test
    y_train_mapped = np.array([label_map[label] for label in y_train])
    y_test_mapped = np.array([label_map[label] for label in y_test])

    # One-hot encode the mapped labels
    y_train_onehot = to_categorical(y_train_mapped, num_classes=num_classes)
    y_test_onehot = to_categorical(y_test_mapped, num_classes=num_classes)

    print(f"One-hot encoded label shape: {y_train_onehot.shape[1]}")

    return X_train_flattened, y_train_onehot, X_test_flattened, y_test_onehot, num_classes, label_map

# Create a simple MLP model for TFLite Micro
def create_mlp_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        # No need for a Flatten layer anymore, input data is already flattened
        tf.keras.layers.Dense(128, activation='relu', input_shape=input_shape),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Train the model
def train_model(model, X_train, y_train, X_val, y_val, epochs=50, batch_size=32):
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(X_train, y_train,
                        validation_data=(X_val, y_val),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[early_stopping],
                        verbose=1)
    return history

# Evaluate the model
def evaluate_model(model, X_test, y_test):
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test accuracy: {accuracy:.4f}")

    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)

    # Convert back to original labels for the classification report
    inv_label_map = {v: k for k, v in label_map.items()}
    y_pred_original = np.array([inv_label_map[label] for label in y_pred_classes])
    y_test_original = np.array([inv_label_map[label] for label in y_test_classes])

    print(classification_report(y_test_original, y_pred_original, digits=5))

# Convert to TFLite
def convert_to_tflite(model, X_test):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8
    converter.inference_output_type = tf.int8

    def representative_dataset_gen():
        for input_value in tf.data.Dataset.from_tensor_slices(X_test).batch(1).take(100):
            yield [input_value]

    converter.representative_dataset = representative_dataset_gen
    tflite_model = converter.convert()

    with open('subject_dependent_mlp_tflite_model.tflite', 'wb') as f:
        f.write(tflite_model)
    print("TFLite model saved as 'subject_dependent_mlp_tflite_model.tflite'")

# Load and preprocess data
X_train, y_train, X_test, y_test, num_classes, label_map = load_and_preprocess_data()
print("num_classes: ", num_classes)

# Split training data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Create and train the MLP model
input_shape = (X_train.shape[1],)  # Input shape is now 1D (900,)
MLP = create_mlp_model(input_shape, num_classes)
history = train_model(MLP, X_train, y_train, X_val, y_val)

# Evaluate the model
evaluate_model(MLP, X_test, y_test)

In [3]:
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(MLP)
tflite_model = converter.convert()

# Save the TFLite model
with open("GeneratedTFLiteFilesAndOGModels/SimpleMLP.tflite", "wb") as f:
    f.write(tflite_model)

print("TensorFlow Lite model has been saved as 'SimpleMLP.tflite'")

# Optional: Save the Keras model
MLP.save("GeneratedTFLiteFilesAndOGModels/SimpleMLP.h5")
print("Keras model has been saved as 'SimpleMLP.h5'")

try:
    from everywhereml.code_generators.tensorflow import convert_model

    c_header = convert_model(MLP, X_test, y_test, model_name="SimpleMLP")

    with open("GeneratedHeaderFiles/SimpleMLP.h", "w") as file:
        file.write(c_header)

    print("C header file has been saved as GeneratedHeaderFiles/SimpleMLP.h'")
except ImportError:
    print("everywhereml library not found. Skipping C header file generation.")

INFO:tensorflow:Assets written to: /tmp/tmp1x9yu6sp/assets


INFO:tensorflow:Assets written to: /tmp/tmp1x9yu6sp/assets


Saved artifact at '/tmp/tmp1x9yu6sp'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100, 9), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 21), dtype=tf.float32, name=None)
Captures:
  126702570761280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126702570764976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126702566066480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126702566070000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126702566067184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126702566071584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126702566067360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126702566073168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126702566066656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126702566074752: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1727419412.253757  231383 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1727419412.253767  231383 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-09-27 12:43:32.253871: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp1x9yu6sp
2024-09-27 12:43:32.254178: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-09-27 12:43:32.254186: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmp1x9yu6sp
2024-09-27 12:43:32.256989: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-09-27 12:43:32.275789: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmp1x9yu6sp
2024-09-27 12:43:32.281145: I tensorflow/cc/saved_model/loader.cc:462] SavedModel load for tags { serve }; Status: success: OK. Took 27276 microseconds.


TensorFlow Lite model has been saved as 'SimpleMLP.tflite'
Keras model has been saved as 'SimpleMLP.h5'
INFO:tensorflow:Assets written to: /tmp/tmpaa1xjlt1/assets


INFO:tensorflow:Assets written to: /tmp/tmpaa1xjlt1/assets


Saved artifact at '/tmp/tmpaa1xjlt1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100, 9), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 21), dtype=tf.float32, name=None)
Captures:
  126702570761280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126702570764976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126702566066480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126702566070000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126702566067184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126702566071584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126702566067360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126702566073168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126702566066656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126702566074752: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1727419412.523286  231383 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1727419412.523296  231383 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-09-27 12:43:32.523402: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpaa1xjlt1
2024-09-27 12:43:32.523715: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-09-27 12:43:32.523723: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpaa1xjlt1
2024-09-27 12:43:32.526502: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-09-27 12:43:32.546216: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpaa1xjlt1
2024-09-27 12:43:32.551843: I tensorflow/cc/saved_model/loader.cc:462] SavedModel load for tags { serve }; Status: success: OK. Took 28443 microseconds.


C header file has been saved as GeneratedHeaderFiles/SimpleMLP.h'
